In [1]:
import tensorflow as tf
physicalDevices = tf.config.list_physical_devices('GPU') 
if physicalDevices:
    tf.config.experimental.set_memory_growth(physicalDevices[0], True)  
from tensorflow import keras
from tensorflow.keras import layers
from data_preprocessor import DataGenerator, GESTURES

import datetime

In [2]:
batch_size=64
epochs=50
input_shape = (20,21,3)
classes = len(GESTURES)
trainGen = DataGenerator('gesture_dataset.h5')

In [3]:
inputs = layers.Input(shape=input_shape)
x = inputs
x = layers.TimeDistributed(layers.Flatten())(x)
x = layers.TimeDistributed(layers.Dropout(0.5))(x)
x = layers.TimeDistributed(layers.Dense(64, activation='relu'))(x)
x = layers.TimeDistributed(layers.Dropout(0.5))(x)
x = layers.TimeDistributed(layers.Dense(64, activation='relu'))(x)
x = layers.TimeDistributed(layers.Dropout(0.3))(x)
x = layers.TimeDistributed(layers.Dense(64, activation='relu'))(x)
x = layers.TimeDistributed(layers.Dropout(0.2))(x)
x = layers.GRU(64)(x)
output = layers.Dense(classes, activation='softmax',name='Output')(x)
model = keras.models.Model(inputs, output)
model.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 20, 21, 3)]       0         
_________________________________________________________________
time_distributed (TimeDistri (None, 20, 63)            0         
_________________________________________________________________
time_distributed_1 (TimeDist (None, 20, 63)            0         
_________________________________________________________________
time_distributed_2 (TimeDist (None, 20, 64)            4096      
_________________________________________________________________
time_distributed_3 (TimeDist (None, 20, 64)            0         
_________________________________________________________________
time_distributed_4 (TimeDist (None, 20, 64)            4160      
_________________________________________________________________
time_distributed_5 (TimeDist (None, 20, 64)            0     

In [4]:
model.fit(trainGen,epochs=epochs)

Epoch 1/50
1080/1080 [==============================] - 9s 6ms/step - loss: 0.7423 - accuracy: 0.7340
Epoch 2/50
1080/1080 [==============================] - 6s 6ms/step - loss: 0.0333 - accuracy: 0.9920
Epoch 3/50
1080/1080 [==============================] - 7s 7ms/step - loss: 0.0235 - accuracy: 0.9942
Epoch 4/50
1080/1080 [==============================] - 7s 7ms/step - loss: 0.0130 - accuracy: 0.9963
Epoch 5/50
1080/1080 [==============================] - 7s 6ms/step - loss: 0.0115 - accuracy: 0.9963
Epoch 6/50
1080/1080 [==============================] - 7s 6ms/step - loss: 0.0096 - accuracy: 0.9975
Epoch 7/50
1080/1080 [==============================] - 7s 6ms/step - loss: 0.0067 - accuracy: 0.9978
Epoch 8/50
1080/1080 [==============================] - 6s 6ms/step - loss: 0.0070 - accuracy: 0.9977
Epoch 9/50
1080/1080 [==============================] - 7s 6ms/step - loss: 0.0037 - accuracy: 0.9989
Epoch 10/50
1080/1080 [==============================] - 7s 6ms/step - loss: 0.005

KeyboardInterrupt: 

In [5]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tfliteModel = converter.convert()
modelPath = "saved_models/MODEL-{}.tflite".format(datetime.datetime.now().strftime("%Y-%m-%d-%H-%M-%S"))
with open(modelPath, 'wb') as f:
  f.write(tfliteModel)

INFO:tensorflow:Assets written to: C:\Users\17ami\AppData\Local\Temp\tmpuzeu8tjk\assets
INFO:tensorflow:Assets written to: C:\Users\17ami\AppData\Local\Temp\tmpuzeu8tjk\assets
